In [ ]:
from sklearn.model_selection import train_test_split
from os import listdir, getcwd
from os.path import join
import matplotlib.pyplot as plt
import numpy as np
from lib import unet
from lib import generator

In [ ]:
DATA_DIR = join(getcwd(), 'data')
TRAIN_DIR = join(DATA_DIR, 'train')
MASK_DIR = join(DATA_DIR, 'train_masks')
TRAIN_IMAGE_LIST = listdir(TRAIN_DIR)
MASK_IMAGE_LIST = listdir(MASK_DIR)
IMG_WIDTH = 320

In [ ]:
train_images, test_images, train_masks, test_masks = train_test_split(TRAIN_IMAGE_LIST, 
                                                                      MASK_IMAGE_LIST, 
                                                                      test_size=0.10000)
train_images, validation_images, train_masks, validation_masks = train_test_split(train_images,
                                                                                  train_masks,
                                                                                  test_size=0.11111)

In [ ]:
train_generator = generator.image_generator(TRAIN_DIR, 
                                            MASK_DIR, 
                                            train_images, 
                                            train_masks, 
                                            batch_size = 10, 
                                            img_dim = [int((1280 / 1918) * IMG_WIDTH), IMG_WIDTH], 
                                            num_colors = 8)

In [ ]:
img, mask = next(train_generator)
plot = 1
for i in range(img.shape[0]):
    plt.subplot(img.shape[0], 3, plot)
    plt.imshow(img[i])
    plot += 1
    plt.subplot(img.shape[0], 3, plot)
    plt.imshow(mask[i])
    plot += 1
    plt.subplot(img.shape[0], 3, plot)
    plt.imshow(img[i])
    plt.imshow(mask[i], alpha=0.5)
    plot += 1
plt.show()

In [ ]:
model = unet.get_unet_model([IMG_WIDTH, IMG_WIDTH, 3], 1e-4)

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=100, epochs=1)